In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import seaborn as sns

In [39]:
# NLP libraries
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# nltk.download('stopwords')
# nltk.download('punkt')
from wordcloud import WordCloud

In [40]:
channel_names

['codebasics',
 'TwoMinutePapers',
 'AINewsOfficial',
 'abhishekkrthakur',
 'IBMTechnology',
 'PROROBOTS']

In [41]:
channel_names = list(json.load(open('../data/external/channel_id.json')).keys())
videos_df = pd.DataFrame()

channel_names.remove('StrataScratch')
channel_names.remove('TheAiGrid')


for channel in channel_names:
    channel_df = pd.read_csv('../data/raw/' + channel + '_videos.csv')
    videos_df = pd.concat([videos_df, channel_df])

videos_df.head()

,playlist_id,video_id,title,published,view_count,like_count,comment_count,duration,definition,tags,default_audio_language,madeforkid,playlist_title,channel_id,channelTitle,playlist_published,playlist_numvideo
0,PLeo1K3hjS3uufBJODNXMY2VuDpFWrDJ3L,in3IB45YEgY,B.Com to Data Analytics Transition,2023-03-18T13:30:10Z,14985,487.0,42,PT29M27S,hd,"['yt:cc=on', 'B.Com to Data Analytics Transiti...",en,False,Non-Tech to Data Analyst Transition Stories,UCh9nVJoWXmFb7sLApWGcLPQ,codebasics,2023-11-17T19:37:00Z,6
1,PLeo1K3hjS3uufBJODNXMY2VuDpFWrDJ3L,Gw6hVMzna7g,Hacks for Moms With a Career Break to Become a...,2023-05-25T13:30:11Z,16163,537.0,55,PT49M2S,hd,"['nurse', 'mom', 'how to be a data analyst wit...",en,False,Non-Tech to Data Analyst Transition Stories,UCh9nVJoWXmFb7sLApWGcLPQ,codebasics,2023-11-17T19:37:00Z,6
2,PLeo1K3hjS3uufBJODNXMY2VuDpFWrDJ3L,4BLxapDqrlA,Mechanical Engineer To Data Analyst,2021-03-11T14:30:13Z,46614,1048.0,65,PT23M58S,hd,"['data analyst', 'michanical engineer to data ...",en,False,Non-Tech to Data Analyst Transition Stories,UCh9nVJoWXmFb7sLApWGcLPQ,codebasics,2023-11-17T19:37:00Z,6
3,PLeo1K3hjS3uufBJODNXMY2VuDpFWrDJ3L,lqEzYDuTnvU,How B com graduate became a data analyst | Dat...,2020-06-15T16:00:09Z,167671,4515.0,339,PT39M16S,hd,"['bcom to data analyst career transition', 'b ...",en,False,Non-Tech to Data Analyst Transition Stories,UCh9nVJoWXmFb7sLApWGcLPQ,codebasics,2023-11-17T19:37:00Z,6
4,PLeo1K3hjS3uufBJODNXMY2VuDpFWrDJ3L,uSLQKYsEYMo,Data Analyst at FORD after 7 years in Mechanic...,2023-08-01T13:30:12Z,15230,398.0,18,PT39M32S,hd,"['yt:cc=on', 'career transition', 'mechanical ...",en,False,Non-Tech to Data Analyst Transition Stories,UCh9nVJoWXmFb7sLApWGcLPQ,codebasics,2023-11-17T19:37:00Z,6


In [42]:
# Remove unnecessary columns
videos_df = videos_df.drop(['playlist_id', 'video_id', 'default_audio_language', 'channel_id'], axis=1)
videos_df.head()

,title,published,view_count,like_count,comment_count,duration,definition,tags,madeforkid,playlist_title,channelTitle,playlist_published,playlist_numvideo
0,B.Com to Data Analytics Transition,2023-03-18T13:30:10Z,14985,487.0,42,PT29M27S,hd,"['yt:cc=on', 'B.Com to Data Analytics Transiti...",False,Non-Tech to Data Analyst Transition Stories,codebasics,2023-11-17T19:37:00Z,6
1,Hacks for Moms With a Career Break to Become a...,2023-05-25T13:30:11Z,16163,537.0,55,PT49M2S,hd,"['nurse', 'mom', 'how to be a data analyst wit...",False,Non-Tech to Data Analyst Transition Stories,codebasics,2023-11-17T19:37:00Z,6
2,Mechanical Engineer To Data Analyst,2021-03-11T14:30:13Z,46614,1048.0,65,PT23M58S,hd,"['data analyst', 'michanical engineer to data ...",False,Non-Tech to Data Analyst Transition Stories,codebasics,2023-11-17T19:37:00Z,6
3,How B com graduate became a data analyst | Dat...,2020-06-15T16:00:09Z,167671,4515.0,339,PT39M16S,hd,"['bcom to data analyst career transition', 'b ...",False,Non-Tech to Data Analyst Transition Stories,codebasics,2023-11-17T19:37:00Z,6
4,Data Analyst at FORD after 7 years in Mechanic...,2023-08-01T13:30:12Z,15230,398.0,18,PT39M32S,hd,"['yt:cc=on', 'career transition', 'mechanical ...",False,Non-Tech to Data Analyst Transition Stories,codebasics,2023-11-17T19:37:00Z,6


In [43]:
import ast

# Function to convert each string to a list using ast.literal_eval
def convert_to_list(string):
    if pd.notna(string):
        return ast.literal_eval(string)
    
    return string

# Convert the tags column to a list
videos_df['tags'] = videos_df['tags'].apply(convert_to_list)
videos_df['tags'].explode().value_counts()

tags
ai                                                        1300
two minute papers                                         1159
deep learning                                              955
machine learning                                           561
technology                                                 444
                                                          ... 
google deepminds deep qlearning playing atari breakout       1
google deepmind atari                                        1
what is deep q learning                                      1
atari ai                                                     1
superhuman atari                                             1
Name: count, Length: 13357, dtype: int64

In [48]:
videos_df['tags'].explode().unique().

array(['yt:cc=on', 'B.Com to Data Analytics Transition', 'career', ...,
       'high technology news', 'robot atlas company boston dynamics',
       'us army'], dtype=object)

In [44]:
videos_df.dtypes

title                  object
published              object
view_count              int64
like_count            float64
comment_count           int64
duration               object
definition             object
tags                   object
madeforkid               bool
playlist_title         object
channelTitle           object
playlist_published     object
playlist_numvideo       int64
dtype: object

In [45]:
videos_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3429 entries, 0 to 164
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               3429 non-null   object 
 1   published           3429 non-null   object 
 2   view_count          3429 non-null   int64  
 3   like_count          3397 non-null   float64
 4   comment_count       3429 non-null   int64  
 5   duration            3429 non-null   object 
 6   definition          3429 non-null   object 
 7   tags                3285 non-null   object 
 8   madeforkid          3429 non-null   bool   
 9   playlist_title      3429 non-null   object 
 10  channelTitle        3429 non-null   object 
 11  playlist_published  3429 non-null   object 
 12  playlist_numvideo   3429 non-null   int64  
dtypes: bool(1), float64(1), int64(3), object(8)
memory usage: 351.6+ KB


In [46]:
videos_df.describe().T

,count,mean,std,min,25%,50%,75%,max
view_count,3429.0,138333.320793,377800.705831,3.0,15194.0,62006.0,146948.0,9656476.0
like_count,3397.0,4662.445687,13043.058262,0.0,469.0,1621.0,5553.0,358793.0
comment_count,3429.0,228.863517,426.645618,0.0,24.0,79.0,277.0,8375.0
playlist_numvideo,3429.0,285.088656,320.332948,1.0,18.0,73.0,548.0,775.0


In [47]:
stop_words = set(stopwords.words('english'))
TwoMinutePapers_df['title_no_stopwords'] = TwoMinutePapers_df['title'].apply(lambda x: [item for item in str(x).split() if item.lower() not in stop_words])

all_words = list([a for b in TwoMinutePapers_df['title_no_stopwords'].tolist() for a in b])
all_words_str = ' '.join(all_words) 

def plot_cloud(wordcloud):
    plt.figure(figsize=(30, 20))
    plt.imshow(wordcloud) 
    plt.axis("off");

wordcloud = WordCloud(width = 2000, height = 1000, random_state=1, background_color='black', 
                      colormap='viridis', collocations=True).generate(all_words_str)
plot_cloud(wordcloud)

NameError: name 'TwoMinutePapers_df' is not defined